In [1]:
import pandas as pd

In [2]:
ground_truth = pd.read_parquet("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/gold/dataset_citation_records_Table.parquet")
ground_truth.columns

Index(['identifier', 'repository', 'citing_publication_link',
       'citation_record_source', 'citation_record_from_doi'],
      dtype='object')

In [3]:
# get excel table from the Articles tab
# ground_truth = pd.read_excel("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/gold/GT_DataLinks_PMC_and_Fenyo.xlsx")
eval_output_latest_mini = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/new-output_full-gpt-4o-mini.csv")
eval_output_latest = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full.csv")
# eval_output_gemma_DAS = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full_Gemma_DAS.csv")
# eval_output_gemma = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full_Gemma.csv")
# eval_output_GPT_DAS = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GPT-DAS.csv")
# eval_output_GPT = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GPT.csv")
# eval_output_Gemini_15 = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GEMINI-1.5.csv")
# eval_output_Gemini_20 = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GEMINI-2.0-exp.csv")
# eval_output_Gemini_15_Pro = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GEMINI-1.5-pro.csv")
# eval_output_GPT_mini_Full = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GPT-4o-mini.csv")
# eval_output_GPT_Full = pd.read_csv("/Users/pietro/Desktop/VIDA-NYU/data-gatherer/output/output_full-GPT-4o.csv")

In [4]:
# from eval drop rows with empty dataset identifier
eval_output_latest_mini.dropna(subset=["dataset_identifier"], inplace=True)
eval_output_latest.dropna(subset=["dataset_id"], inplace=True)
# eval_output_gemma_DAS.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_gemma.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_GPT_DAS.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_GPT.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_Gemini_15.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_Gemini_20.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_Gemini_15_Pro.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_GPT_mini_Full.dropna(subset=["dataset_identifier"], inplace=True)
# eval_output_GPT_Full.dropna(subset=["dataset_identifier"], inplace=True)

In [5]:
# keep only columns of interes
eval_output_latest_mini = eval_output_latest_mini[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
eval_output_latest = eval_output_latest[['dataset_id', 'repository_reference', 'source_url']]
# eval_output_gemma_DAS = eval_output_gemma_DAS[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_gemma = eval_output_gemma[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_GPT_DAS = eval_output_GPT_DAS[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_GPT = eval_output_GPT[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_Gemini_15 = eval_output_Gemini_15[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_Gemini_20 = eval_output_Gemini_20[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_Gemini_15_Pro = eval_output_Gemini_15_Pro[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_GPT_mini_Full = eval_output_GPT_mini_Full[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]
# eval_output_GPT_Full = eval_output_GPT_Full[["source_url", "dataset_identifier", "dataset_webpage", "data_repository"]]

In [9]:
def calculate_performance_metrics(ground_truth_df: pd.DataFrame, eval_df: pd.DataFrame) -> dict:
    """
    Calculate performance metrics (precision, recall, F1-score) for dataset extraction,
    considering exact, partial, and alias matches. Includes verbose debugging output.
    """
    # Initialize counters for total scores
    total_precision = 0
    total_recall = 0
    num_sources = ground_truth_df['citing_publication_link'].nunique()

    print(f"Number of unique source pages: {num_sources}")

    # Iterate through each unique source page'identifier','repository','citing_publications_links
    for source_page in ground_truth_df['citing_publication_link'].unique():
        print(f"\nProcessing source page: {source_page}")

        # Extract ground truth datasets for this source page
        gt_data = ground_truth_df[ground_truth_df['citing_publication_link'] == source_page]
        gt_datasets = set(gt_data['identifier'].dropna().str.lower())
        # gt_webpages = set(gt_data['Dataset Webpage'].dropna().str.lower())
        gt_repositories = set(gt_data['repository'].dropna().str.lower())

        print(f"Ground truth datasets: {gt_datasets}")
        # print(f"Ground truth webpages: {gt_webpages}")
        print(f"Ground truth repositories: {gt_repositories}")

        # Extract evaluation datasets for this source page
        eval_data = eval_df[eval_df['source_url'] == source_page]
        eval_datasets = set(eval_data['dataset_id'].dropna().str.lower())
        # eval_webpages = set(eval_data['dataset_webpage'].dropna().str.lower())
        eval_repositories = set(eval_data['repository_reference'].dropna().str.lower())

        print(f"Evaluation datasets: {eval_datasets}")
        # print(f"Evaluation webpages: {eval_webpages}")
        print(f"Evaluation repositories: {eval_repositories}")

        # Handle cases where both ground truth and evaluation are empty
        if not gt_datasets and not eval_datasets:
            print("No datasets in both ground truth and evaluation. Perfect precision and recall.")
            total_precision += 1
            total_recall += 1
            continue

        # Initialize match counters
        exact_matches = 0
        partial_matches = 0
        false_positives = 0
        false_negatives = 0

        # Track matches to avoid double counting
        matched_gt = set()
        matched_eval = set()

        # Check for exact matches first
        for eval_id in eval_datasets:
            if eval_id in gt_datasets:
                print(f"Exact match found: {eval_id}")
                exact_matches += 1
                matched_gt.add(eval_id)
                matched_eval.add(eval_id)

        # Check for partial matches and aliases
        for eval_id in eval_datasets - matched_eval:
            for gt_id in gt_datasets - matched_gt:
                if eval_id in gt_id or gt_id in eval_id:  # Partial match or alias
                    print(f"Partial or alias match found: eval_id={eval_id}, gt_id={gt_id}")
                    partial_matches += 1
                    matched_gt.add(gt_id)
                    matched_eval.add(eval_id)
                    break

        # Calculate False Positives (remaining unmatched eval datasets)
        FP = eval_datasets - matched_eval

        # Remove aliases from False Positives
        for eval_id in list(FP):
            for matched_id in matched_gt:
                if eval_id in matched_id or matched_id in eval_id:  # Alias detected
                    print(f"Removing alias from false positives: eval_id={eval_id}, matched_id={matched_id}")
                    FP.discard(eval_id)
                    break

        false_positives = len(FP)
        print(f"False positives: {FP}")

        # Calculate False Negatives (remaining unmatched ground truth datasets)
        FN = gt_datasets - matched_gt
        false_negatives = len(FN)
        print(f"False negatives: {FN}")

        # Calculate precision and recall for this source page
        true_positives = exact_matches + partial_matches
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

        print(f"Precision for {source_page}: {precision}")
        print(f"Recall for {source_page}: {recall}")

        # Accumulate totals
        total_precision += precision
        total_recall += recall

    # Calculate overall metrics
    average_precision = total_precision / num_sources if num_sources > 0 else 0
    average_recall = total_recall / num_sources if num_sources > 0 else 0
    f1_score = 2 * (average_precision * average_recall) / (average_precision + average_recall) if (average_precision + average_recall) > 0 else 0

    return {
        "average_precision": average_precision,
        "average_recall": average_recall,
        "f1_score": f1_score
    }

In [13]:
# only use rows that are also in eval_output_latest
ground_truth = ground_truth[ground_truth['citing_publication_link'].isin(eval_output_latest['source_url'])]
len(ground_truth)

1248

In [14]:
metrics_latest = calculate_performance_metrics(ground_truth, eval_output_latest)

Number of unique source pages: 781

Processing source page: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10618727
Ground truth datasets: {'gse235080', 'pxd038691'}
Ground truth repositories: {'geo', 'pride'}
Evaluation datasets: {'pxd038691'}
Evaluation repositories: {'pride'}
Exact match found: pxd038691
False positives: set()
False negatives: {'gse235080'}
Precision for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10618727: 1.0
Recall for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10618727: 0.5

Processing source page: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11399395
Ground truth datasets: {'gse254680', 'gse254679', 'gse254674', 'pxd048847', 'gse254681', 'gse254673'}
Ground truth repositories: {'geo', 'pride'}
Evaluation datasets: {'pxd048847'}
Evaluation repositories: {'pride'}
Exact match found: pxd048847
False positives: set()
False negatives: {'gse254680', 'gse254679', 'gse254674', 'gse254681', 'gse254673'}
Precision for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1139939

In [15]:
# Output the results
print("Performance Metrics:")
print(f"Average Precision: {metrics_latest['average_precision']:.4f}")
print(f"Average Recall: {metrics_latest['average_recall']:.4f}")
print(f"F1-Score: {metrics_latest['f1_score']:.4f}")

Performance Metrics:
Average Precision: 0.8668
Average Recall: 0.6621
F1-Score: 0.7508


In [ ]:
#metrics_gemma_DAS = calculate_performance_metrics(ground_truth, eval_output_gemma_DAS)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_gemma_DAS['average_precision']:.4f}")
# print(f"Average Recall: {metrics_gemma_DAS['average_recall']:.4f}")
# print(f"F1-Score: {metrics_gemma_DAS['f1_score']:.4f}")

In [ ]:
# metrics_gemma = calculate_performance_metrics(ground_truth, eval_output_gemma)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_gemma['average_precision']:.4f}")
# print(f"Average Recall: {metrics_gemma['average_recall']:.4f}")
# print(f"F1-Score: {metrics_gemma['f1_score']:.4f}")

In [ ]:
# metrics_GPT_DAS = calculate_performance_metrics(ground_truth, eval_output_GPT_DAS)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_GPT_DAS['average_precision']:.4f}")
# print(f"Average Recall: {metrics_GPT_DAS['average_recall']:.4f}")
# print(f"F1-Score: {metrics_GPT_DAS['f1_score']:.4f}")

In [ ]:
# metrics_GPT = calculate_performance_metrics(ground_truth, eval_output_GPT)#

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_GPT['average_precision']:.4f}")
# print(f"Average Recall: {metrics_GPT['average_recall']:.4f}")
# print(f"F1-Score: {metrics_GPT['f1_score']:.4f}")

In [ ]:
# metrics_Gemini_15 = calculate_performance_metrics(ground_truth, eval_output_Gemini_15)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_Gemini_15['average_precision']:.4f}")
# print(f"Average Recall: {metrics_Gemini_15['average_recall']:.4f}")
# print(f"F1-Score: {metrics_Gemini_15['f1_score']:.4f}")

In [ ]:
# metrics_Gemini_20 = calculate_performance_metrics(ground_truth, eval_output_Gemini_20)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_Gemini_20['average_precision']:.4f}")
# print(f"Average Recall: {metrics_Gemini_20['average_recall']:.4f}")
# print(f"F1-Score: {metrics_Gemini_20['f1_score']:.4f}")

In [ ]:
# metrics_Gemini_15_Pro = calculate_performance_metrics(ground_truth, eval_output_Gemini_15_Pro)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_Gemini_15_Pro['average_precision']:.4f}")
# print(f"Average Recall: {metrics_Gemini_15_Pro['average_recall']:.4f}")
# print(f"F1-Score: {metrics_Gemini_15_Pro['f1_score']:.4f}")

In [ ]:
# metrics_GPT_mini_Full = calculate_performance_metrics(ground_truth, eval_output_GPT_mini_Full)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_GPT_mini_Full['average_precision']:.4f}")
# print(f"Average Recall: {metrics_GPT_mini_Full['average_recall']:.4f}")
# print(f"F1-Score: {metrics_GPT_mini_Full['f1_score']:.4f}")

In [ ]:
# metrics_GPT_Full = calculate_performance_metrics(ground_truth, eval_output_GPT_Full)

In [ ]:
# # Output the results
# print("Performance Metrics:")
# print(f"Average Precision: {metrics_GPT_Full['average_precision']:.4f}")
# print(f"Average Recall: {metrics_GPT_Full['average_recall']:.4f}")
# print(f"F1-Score: {metrics_GPT_Full['f1_score']:.4f}")